In [ ]:
import numpy as np
import pandas as pd
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,  MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_dev_image_path = glob.glob('train/*.jpg')
train_dev_coord_csv = pd.read_csv("train.csv", index_col="id")

In [ ]:
train_dev_images = []
train_dev_coords = []
for path in tqdm(train_dev_image_path):
    img = image.load_img(path, target_size=(200, 200))
    img = image.img_to_array(img)
    train_dev_images.append(img)
    x, y = train_dev_coord_csv.loc[path[-13:-4]]
    train_dev_coords.append((x, y))

In [ ]:
# Plot the coords distribution
# Not nesserary to run to get the output
coords = sorted(train_dev_coords)
x = [a for a,_ in coords]
y = [b for _,b in coords]
plt.plot(x, y,'o', ms=0.5)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
train_dev_images = np.array(train_dev_images)
train_dev_coords = np.array(train_dev_coords)

In [ ]:
pretrain_model = ResNet50(weights="imagenet", input_shape=[200,200,3],include_top=False)
for layer in pretrain_model.layers:
    layer.trainable = False
conv1 = Conv2D(filters = 32, kernel_size = (1,1), padding = 'same', activation = 'relu')(pretrain_model.layers[-1].output)
conv2 = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu')(conv1)
conv3 = Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu')(conv2)
max_pooling = MaxPooling2D(pool_size= (2,2), strides = 2, padding = 'same')(conv3)
flat1 = Flatten()(max_pooling)
dropout = Dropout(0.3)(flat1)
class1 = Dense(1048, activation='relu')(dropout)
output = Dense(2)(class1)
model = Model(inputs=pretrain_model.inputs, outputs=output)

In [ ]:
x_train = preprocess_input(train_dev_images) 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
           loss="mean_absolute_error")
model.fit(x=x_train, y=train_dev_coords,
          validation_split = 0.2,
    epochs=200, batch_size=128)

In [ ]:
test_image_path = sorted(glob.glob('test/*.jpg'))
test_images = []
for path in tqdm(test_image_path):
    img = image.load_img(path, target_size=(200, 200))
    img = image.img_to_array(img)
    test_images.append(img)
test_images = np.array(test_images)
test_images = preprocess_input(test_images) 
predict = model.predict(test_images)

In [ ]:
all_test_coord = []
for i, test_name in enumerate(test_image_path):
  curr = []
  test_name = test_name[-13:-4]
  curr.append(test_name)
  curr.append(predict[i][0])
  curr.append(predict[i][1])
  all_test_coord.append(curr)


In [ ]:
df = pd.DataFrame(all_test_coord, columns = ["id", 'x', 'y'])
df.to_csv("output_cnn.csv", index=False)